In [1]:
import requests
import json
from datetime import datetime
import time

In [2]:
def fetch_json(url):
    try: 
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error fetching JSON from {url}: {e}")
        return {}

In [13]:
BASE_URL = "https://gbfs.velobixi.com/gbfs/gbfs.json"
STATION_STATUS = "station_status"
MINIMUM_WAIT = 2 # second


In [4]:
def retreive_urls():
    urls = fetch_json(BASE_URL)
    return find_url_bixi(urls,STATION_STATUS)

def find_url_bixi(urls,name):
    for item in urls["data"]["en"]["feeds"]:
        if item["name"]==name:
            return item["url"]

In [5]:
station_status_url = retreive_urls()


In [6]:
station_status = fetch_json(station_status_url)

In [20]:
station_status["data"]["stations"][0]

{'station_id': '1',
 'num_bikes_available': 18,
 'num_ebikes_available': 14,
 'num_bikes_disabled': 0,
 'num_docks_available': 1,
 'num_docks_disabled': 0,
 'is_installed': 1,
 'is_renting': 1,
 'is_returning': 1,
 'last_reported': 1748550391,
 'eightd_has_available_keys': False,
 'is_charging': False}

In [7]:
station_status["ttl"]

10

In [ ]:
station_status = fetch_json(station_status_url)
now = 
towait = (station_status["last_updated"] + (station_status["ttl"])) - datetime.now().timestamp()
if towait < 0:
    towait = 0
towait += MINIMUM_WAIT
print(towait)
time.sleep(towait)
station_status2 = fetch_json(station_status_url)
if(station_status2["last_updated"] != station_status["last_updated"]):
    add_to_average(average,station_status)

9.864460945129395
yay


In [ ]:
from collections import defaultdict


In [ ]:
FIELDS = ['num_bikes_available','num_ebikes_available','num_bikes_disabled','num_docks_available','num_docks_disabled','is_installed','is_renting','is_returning']
MINIMUM_WAIT = 2

In [22]:
def floor_to_previous_15min(posix_timestamp):
    interval = 15 * 60  # 15mn
    floored_timestamp = (posix_timestamp // interval) * interval
    return floored_timestamp

In [ ]:
def add_to_agg(agregate_holder,station_status_json):
    for station in station_status_json["data"]["stations"]:
        for field in FIELDS:
            agregate_holder[station["station_id"]][field] += station[field]


In [36]:
import math
def compute_ceiled_means(aggregate_holder, nb_updates,start_time):
    results = []
    for station_id, field_sums in aggregate_holder.items():
        agg = {
            "station_id": station_id,
            "timestamp": start_time
        }

        for field, total in field_sums.items():
            mean = total / nb_updates
            agg[field] = math.ceil(mean)
        results.append(agg)

    return results

Sleeping ...
adding to aggregate
Sleeping ...
adding to aggregate
Sleeping ...
adding to aggregate
Sleeping ...
adding to aggregate
Sleeping ...
Sleeping ...
adding to aggregate
Sleeping ...
Sleeping ...
oho


In [40]:
import psycopg2
from dotenv import load_dotenv
import os

load_dotenv()

def get_pg_connection():
    return psycopg2.connect(
        user=os.getenv("user"),
        password=os.getenv("password"),
        host=os.getenv("host"),
        port=os.getenv("port"),
        dbname=os.getenv("dbname")
    )   

In [ ]:
from psycopg2.extras import execute_batch
def insert_ceiled_means(dict_rows):
    rows = [
        (
            row["station_id"],
            row["timestamp"],  # integer UNIX timestamp
            row["num_bikes_available"],
            row["num_ebikes_available"],
            row["num_bikes_disabled"],
            row["num_docks_available"],
            row["num_docks_disabled"]
        )
        for row in dict_rows
    ]

    insert_query = """
        INSERT INTO bixi_dispo (
            station_id, timestamp,
            num_bikes_available, num_ebikes_available,
            num_bikes_disabled, num_docks_available, num_docks_disabled
        )
        VALUES (
            %s,
            TO_TIMESTAMP(%s),
            %s, %s, %s, %s, %s
        )
    """

    try:
        conn = get_pg_connection()
        cur = conn.cursor()
        cur.execute("SET TIME ZONE 'America/Montreal';")
        execute_batch(cur, insert_query, rows)
        conn.commit()
        cur.close()
        conn.close()
        print("Inserted", len(rows), "rows.")
    except Exception as e:
        print("Failed to insert:", e)


In [ ]:
station_status = fetch_json(station_status_url)
while(True):
    agregate_holder = defaultdict(lambda: defaultdict(int))
    nb_updates = 0
    start_time = floor_to_previous_15min(station_status["last_updated"])
    max_time = start_time + 15 * 60 # 15mn
    #max_time = station_status["last_updated"] + 60 # 15mn
    add_to_agg(agregate_holder,nb_updates,station_status)
    nb_updates+=1
    towait = (station_status["last_updated"] + (station_status["ttl"])) - datetime.now().timestamp()
    if(towait < 0):
        towait = 0
    towait +=MINIMUM_WAIT
    time.sleep(towait)
    prev_timestamp = station_status["last_updated"]
    while True:
        u_station_status = fetch_json(station_status_url)
        if u_station_status["last_updated"] > max_time:
            station_status = u_station_status
            print("Finishing the 15mn")
            break
        if u_station_status["last_updated"] != prev_timestamp :
            print(".",end="")
            add_to_agg(agregate_holder,nb_updates,station_status) 
            nb_updates+=1
            prev_timestamp = u_station_status["last_updated"] 
        towait = (u_station_status["last_updated"] + (u_station_status["ttl"])) - datetime.now().timestamp()
        towait += MINIMUM_WAIT
        if(towait < 0):
            towait = 0
        time.sleep(towait)
    to_send = compute_ceiled_means(agregate_holder,nb_updates,start_time)
    insert_ceiled_means(to_send)

.......................Finishing the 15mn
Inserted 959 rows.
.....................................................................Finishing the 15mn
Inserted 959 rows.
